## 1\Setup

In [1]:
%pip install openai
%pip install pandas
%pip install scikit-learn
%pip install matplotlib

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install kagglehub

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import ast
import pandas as pd
import numpy as np
import kagglehub
import random
from copy import deepcopy

from openai import OpenAI
import os
import re
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import json
import matplotlib

In [4]:
datagen_model = "gpt-4o"
api_key = '<yourkey>' 

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", api_key))

BatchSize = 5 # For each gpt request, access BatchSize misconceptions

## 2\MisconceptionName Mapping

In [5]:
# data_dir = "/kaggle/input/eedi-mining-misconceptions-in-mathematics"
data_dir = "D:\VSCode\Kaggle_eedi_sorted\data"
df = pd.read_csv(os.path.join(data_dir, "train.csv"))
content_df = pd.read_csv(os.path.join(data_dir, "misconception_mapping.csv"))

content_df["MisconceptionId"] = content_df["MisconceptionId"].astype(str)
id2name = dict(zip(content_df["MisconceptionId"], content_df["MisconceptionName"]))

In [6]:
misconception_ids = set()
for col in ["MisconceptionAId", "MisconceptionBId", "MisconceptionCId", "MisconceptionDId"]:
    misconception_ids.update(df[col].dropna().astype(int).unique())
misconception_ids = list(map(str, misconception_ids))
print(f"Number of unique MisconceptionIds: {len(misconception_ids)}")

Number of unique MisconceptionIds: 1604


In [7]:
missing_ids = content_df[~content_df["MisconceptionId"].isin(misconception_ids)]["MisconceptionId"].unique().tolist()
len(missing_ids)

983

In [8]:
all_ids = missing_ids + misconception_ids
len(all_ids)

2587

In [9]:
misconception_ids = set()
for col in ["MisconceptionAId", "MisconceptionBId", "MisconceptionCId", "MisconceptionDId"]:
    misconception_ids.update(df[col].dropna().astype(int).unique())
misconception_ids = list(map(str, misconception_ids))
print(f"Number of unique MisconceptionIds: {len(misconception_ids)}")

Number of unique MisconceptionIds: 1604


In [10]:
df = df.join(content_df,on='MisconceptionAId')
df = df.drop('MisconceptionId',axis=1)
df.columns.values[-1] = 'MisconceptionAName'
df = df.join(content_df,on='MisconceptionBId')
df = df.drop('MisconceptionId',axis=1)
df.columns.values[-1] = 'MisconceptionBName'
df = df.join(content_df,on='MisconceptionCId')
df = df.drop('MisconceptionId',axis=1)
df.columns.values[-1] = 'MisconceptionCName'
df = df.join(content_df,on='MisconceptionDId')
df = df.drop('MisconceptionId',axis=1)
df.columns.values[-1] = 'MisconceptionDName'

## 3\ misconception clustering

## 4\prompt and data

In [50]:
def gen_reference_mcq(qid_list,df):
    qlist = []
    for q in qid_list:
        SubjectName = df.loc[q,'SubjectName']
        ConstructName = df.loc[q,'ConstructName']
        QuestionText = df.loc[q,'QuestionText']
        AnswerAText = df.loc[q,'AnswerAText']
        AnswerBText = df.loc[q,'AnswerBText']
        AnswerCText = df.loc[q,'AnswerCText']
        AnswerDText = df.loc[q,'AnswerDText']
        CorrectAnswer = df.loc[q,'CorrectAnswer']
        # A_ifcorrect = "correct" if df.loc[q, 'CorrectAnswer'] == 'A' else "incorrect"
        # B_ifcorrect = "correct" if df.loc[q, 'CorrectAnswer'] == 'B' else "incorrect"
        # C_ifcorrect = "correct" if df.loc[q, 'CorrectAnswer'] == 'C' else "incorrect"
        # D_ifcorrect = "correct" if df.loc[q, 'CorrectAnswer'] == 'D' else "incorrect"
        MisconceptionA = "No matched misconception" if pd.isna(df.loc[q, 'MisconceptionAName']) else "Correct answer" if df.loc[q,'CorrectAnswer']=='A' else df.loc[q,'MisconceptionAName']
        MisconceptionB = "No matched misconception" if pd.isna(df.loc[q, 'MisconceptionBName']) else "Correct answer" if df.loc[q,'CorrectAnswer']=='B' else df.loc[q,'MisconceptionBName']
        MisconceptionC = "No matched misconception" if pd.isna(df.loc[q, 'MisconceptionCName']) else "Correct answer" if df.loc[q,'CorrectAnswer']=='C' else df.loc[q,'MisconceptionCName']
        MisconceptionD = "No matched misconception" if pd.isna(df.loc[q, 'MisconceptionDName']) else "Correct answer" if df.loc[q,'CorrectAnswer']=='D' else df.loc[q,'MisconceptionDName']
        MisconceptionAId = int(df.loc[q,'MisconceptionAId']) if not pd.isna(df.loc[q,'MisconceptionAId']) else -1 
        MisconceptionBId = int(df.loc[q,'MisconceptionBId']) if not pd.isna(df.loc[q,'MisconceptionBId']) else -1 
        MisconceptionCId = int(df.loc[q,'MisconceptionCId']) if not pd.isna(df.loc[q,'MisconceptionCId']) else -1 
        MisconceptionDId = int(df.loc[q,'MisconceptionDId']) if not pd.isna(df.loc[q,'MisconceptionDId']) else -1 
        reference_mcq = f''' Question {q}
Subject: {SubjectName}  Construct: {ConstructName}
Question: {QuestionText}
AnswerA: {AnswerAText}
AnswerB: {AnswerBText}
AnswerC: {AnswerCText}
AnswerD: {AnswerDText}
CorrectAnswer: {CorrectAnswer}
Misconception A: {MisconceptionA}
Misconception B: {MisconceptionB}
Misconception C: {MisconceptionC}
Misconception D: {MisconceptionD}
        '''
        qlist.append(reference_mcq)
    return '\n'.join(qlist)

In [51]:
def get_res(mids,rids,df): # mids is a list of missing misconceptionid, rids is a list of reference mid
    rows = []
    for i in mids:
        rows.append('Misconception:' +content_df.loc[int(i),'MisconceptionName'])
    cluster_misconceptions = '\n'.join(rows)
    reference_mcqs = gen_reference_mcq(rids,df) #input
    num_mcqs = len(mids) # generate 2 questions for each misconception
    
    prompt = f"""You are a Mathematics teacher. Your task is to generate Multiple Choice Questions (MCQs) that diagnose the following misconceptions:
    <misconceptions>
    {cluster_misconceptions}
    </misconceptions>

    Here are reference MCQs that demonstrate the questions and misconceptions:
    <reference_mcqs>
    {reference_mcqs}
    </reference_mcqs>
    
    First, analyze the reference MCQs carefully:
    1. Understand how to derive the correct answer and how the wrong answers map to respective misconceptions
    2. Note the style, difficulty level, and precision of language used
    3. List misconceptions still need coverage
    
    Then, genrate {num_mcqs} new MCQs that diagnose misconceptions not already covered by the reference MCQs.
    For each needed misconception, in your <reasoning> tag, show your process following these guidelines:
    1. brainstorm mathematical contexts where it commonly appears
    2. solve the probelm yourself and then design wrong answers leading to specific miesconceptions
    3. Make questions challenging enough that students must demonstrate real understanding
    4. ensure the answers are plausible and stem from genuine misconceptions, not careless errors
    5. Use precise mathematical language matching the style of reference MCQs
    5. Keep the construct name and subject name as short as possible
    
    """
    response = client.chat.completions.create(
    model=datagen_model,
    messages=[
        {"role": "system", "content": "You are a helpful assistant designed to generate synthetic data."},
        {"role": "user", "content": prompt}
      ]
    )
    res = response.choices[0].message.content
    return res


In [52]:
def extract_data(res):
    constructs = re.findall(r"Construct:\s*(.*?)\n", res)
    subjects = re.findall(r"Subject:\s*(.*?)\n", res)
    correct_answers = re.findall(r"CorrectAnswer:\s*(.*?)\n", res)
    questions = re.findall(r"Question:\s*(.*?)\n", res)
    AnswerA = re.findall(r"AnswerA:(.*?)\n", res)
    AnswerB = re.findall(r"AnswerB:(.*?)\n", res)
    AnswerC = re.findall(r"AnswerC:(.*?)\n", res)    # Create a DataFrame
    AnswerD = re.findall(r"AnswerD:(.*?)\n", res)
    MisconceptionA = re.findall(r"MisconceptionA is *(.*?):", res)
    MisconceptionB = re.findall(r"MisconceptionB is *(.*?):", res)
    MisconceptionC = re.findall(r"MisconceptionC is *(.*?):", res)    # Create a DataFrame
    MisconceptionD = re.findall(r"MisconceptionD is *(.*?):", res)
    df = pd.DataFrame({
        'ConstructId': constructs,
        'SubjectName': subjects,
        'CorrectAnswer': correct_answers,
        'QuestionText': questions,
        'AnswerAText': AnswerA,
        'AnswerBText': AnswerB,
        'AnswerCText': AnswerC,
        'AnswerDText': AnswerD,
        'MisconceptionAId': MisconceptionA,
        'MisconceptionBId': MisconceptionB,
        'MisconceptionCId': MisconceptionC,
        'MisconceptionDId': MisconceptionD,
    })
    return df

## 5\ synthesize data

In [53]:
i = 0
res_list = []
df_list = []
misconception_ids = [int(x) for x in misconception_ids]
missing_ids = [int(x) for x in missing_ids]

while i<=len(missing_ids)//BatchSize:
    mids = missing_ids[BatchSize*i:BatchSize*(i+1)] if BatchSize*(i+1)<=len(missing_ids) else len(missing_ids)
    rids = misconception_ids[BatchSize*i:BatchSize*(i+1)]
    res = get_res(mids,rids,df)
    res_list.append(res)
    syn_df = extract_data(res)
    df_list.append(syn_df)
    i += BatchSize
    # break
syn_dfs = pd.concat(df_list)

ValueError: All arrays must be of the same length

In [15]:
content_df['MisconceptionId']=content_df['MisconceptionId'].astype('int64')
syn_dfs['MisconceptionAId']=syn_dfs['MisconceptionAId'].astype('int64')
syn_dfs['MisconceptionBId']=syn_dfs['MisconceptionBId'].astype('int64')
syn_dfs['MisconceptionCId']=syn_dfs['MisconceptionCId'].astype('int64')
syn_dfs['MisconceptionDId']=syn_dfs['MisconceptionDId'].astype('int64')

In [16]:
syn_dfs = syn_dfs.join(content_df,on='MisconceptionAId')
syn_dfs = syn_dfs.drop('MisconceptionId',axis=1)
syn_dfs.columns.values[-1] = 'MisconceptionAName'
syn_dfs = syn_dfs.join(content_df,on='MisconceptionBId')
syn_dfs = syn_dfs.drop('MisconceptionId',axis=1)
syn_dfs.columns.values[-1] = 'MisconceptionBName'
syn_dfs = syn_dfs.join(content_df,on='MisconceptionCId')
syn_dfs = syn_dfs.drop('MisconceptionId',axis=1)
syn_dfs.columns.values[-1] = 'MisconceptionCName'
syn_dfs = syn_dfs.join(content_df,on='MisconceptionDId')
syn_dfs = syn_dfs.drop('MisconceptionId',axis=1)
syn_dfs.columns.values[-1] = 'MisconceptionDName'

In [17]:
syn_dfs.to_csv('synthesized_data_gpt_4o.csv')

In [47]:
# # debug
# rows = []
# i = 0
# mids = missing_ids[BatchSize*i:BatchSize*(i+1)] if BatchSize*(i+1)<=len(missing_ids) else len(missing_ids)
# rids = misconception_ids[BatchSize*i:BatchSize*(i+1)]
# for i in mids:
#     rows.append('Misconception:' +content_df.loc[int(i),'MisconceptionName'])
# cluster_misconceptions = '\n'.join(rows)
# reference_mcqs = gen_reference_mcq(rids,df) #input
# num_mcqs = len(mids) # generate 2 questions for each misconception

# prompt = f"""You are a Mathematics teacher. Your task is to generate Multiple Choice Questions (MCQs) that diagnose the following misconceptions:
# <misconceptions>
# {cluster_misconceptions}
# </misconceptions>

# Here are reference MCQs that demonstrate the questions and misconceptions:
# <reference_mcqs>
# {reference_mcqs}
# </reference_mcqs>

# First, analyze the reference MCQs carefully:
# 1. Understand how to derive the correct answer and how the wrong answers map to respective misconceptions
# 2. Note the style, difficulty level, and precision of language used
# 3. List misconceptions still need coverage

# Then, genrate {num_mcqs} new MCQs that diagnose misconceptions not already covered by the reference MCQs.
# For each needed misconception, in your <reasoning> tag, show your process following these guidelines:
# 1. brainstorm mathematical contexts where it commonly appears
# 2. solve the probelm yourself and then design wrong answers leading to specific miesconceptions
# 3. Make questions challenging enough that students must demonstrate real understanding
# 4. ensure the answers are plausible and stem from genuine misconceptions, not careless errors
# 5. Use precise mathematical language matching the style of reference MCQs
# 5. Keep the construct name and subject name as short as possible

# """
# response = client.chat.completions.create(
# model=datagen_model,
# messages=[
#     {"role": "system", "content": "You are a helpful assistant designed to generate synthetic data."},
#     {"role": "user", "content": prompt}
#     ]
# )
# res = response.choices[0].message.content

In [48]:
# print(prompt)

You are a Mathematics teacher. Your task is to generate Multiple Choice Questions (MCQs) that diagnose the following misconceptions:
<misconceptions>
Misconception:When measuring a reflex angle, gives the acute or obtuse angle that sums to 360 instead
Misconception:Believes gradient = change in y
Misconception:Believes duration can be read from a timetable, rather than calculating from start and end times
Misconception:Thinks that the side view does not include the furthest rows back
Misconception:Does not subtract from the hours, when having to borrow for a time calculation
</misconceptions>

Here are reference MCQs that demonstrate the questions and misconceptions:
<reference_mcqs>
 Question 0
Subject: BIDMAS  Construct: Use the order of operations to carry out calculations involving powers
Question: \[
3 \times 2+4-5
\]
Where do the brackets need to go to make the answer equal \( 13 \) ?
AnswerA: \( 3 \times(2+4)-5 \)
AnswerB: \( 3 \times 2+(4-5) \)
AnswerC: \( 3 \times(2+4-5) \)
An

In [49]:
# print(res)

<Question 5>
Subject: Measuring Angles  Construct: Measure reflex angles
Question: What is the measure of the reflex angle if the angle between the two lines is \( 110^\circ \)?
AnswerA: \( 250^\circ \)
AnswerB: \( 70^\circ \)
AnswerC: \( 110^\circ \)
AnswerD: \( 180^\circ \)
CorrectAnswer: A.
Misconception A: Gives the acute angle instead of the reflex by subtracting from \( 180^\circ \) (Answer B)
Misconception B: Gives the angle as it is, without calculating the reflex angle (Answer C)
Misconception C: Confuses the concepts of supplementary and reflex angles (Answer D)

<reasoning>
1. Reflex angles are those greater than \(180^\circ\). A common mistake happens when students confuse this with the smaller angle within \(180^\circ\).
2. The correct reflex angle should sum to \(360^\circ\) with the known angle: \(360 - 110 = 250\).
3. I designed the incorrect answers to map to key misconceptions: B as confusion with acute or obtuse, C as literal misinterpretation, D as misconception wit